In [1]:
import requests
import pandas as pd
import time
import json

In [60]:
# using mymemory to generate the Arabic Training Data
def translate(text):
        
        url = "https://api.mymemory.translated.net/get"
        params = {
            "q": text,
            "langpair": "en|ar",
            "key": "43a221557ce746042181",
            "de": "abdohefney1@gmail.com"
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if data.get("responseStatus") != 200:
            raise Exception(f"MyMemory error: {data.get('responseDetails', 'Unknown error')}")
        
        return data["responseData"]["translatedText"]

def translate_google(text):
        """Translate using Google Translate"""
        url = "https://translate.googleapis.com/translate_a/single"
        params = {
            "client": "gtx",
            "sl": "en",
            "tl": "ar",
            "dt": "t",
            "q": text
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        # Parse the JSON response
        data = json.loads(response.text)
        if not data or not data[0]:
            raise Exception("No translation returned from Google")
        
        # Extract translation from nested JSON
        translation_parts = [part[0] for part in data[0] if part[0]]
        return "".join(translation_parts)


In [67]:
data = pd.read_csv("./translated_data.csv")
not_translated = data[data["arabic_instruction"].isna()]
not_translated.reset_index(inplace=True)

In [114]:
data = pd.read_json("C:\\Users\\abdo_\\Downloads\\Translation_dataset\\test.json")
data = data["question"]
arabic = pd.Series([""]*data.shape[0])

data = pd.concat([arabic,data],axis=1).rename(columns={0:"arabic_instruction","question":"instruction"})

In [115]:
# Translate it
# This bad implementation because i will get rate limited if i used 
# the SpiderData.map(translate)
for i in range(data.shape[0]):
    try:
        data.loc[i,"arabic_instruction"] = translate_google(data.loc[i,"instruction"])
    except:
        print("error occured")
        break

In [116]:
data.to_csv("./test_data.csv",index=False,header=True,encoding="utf-8")